In [2]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
import json
from PIL import ImageDraw
from copy import copy
from PIL import ImageFont
import time
font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 15)
os.environ["CUDA_VISIBLE_DEVICES"] = ''

2023-07-28 11:09:02.476920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 11:09:03.164405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

model = K.models.load_model('../../model/skip/MobileNetV2_checkpoints.h5')

2023-07-28 11:09:05.862663: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-28 11:09:05.862708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gil
2023-07-28 11:09:05.862714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gil
2023-07-28 11:09:05.862847: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-07-28 11:09:05.862870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-07-28 11:09:05.862873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


In [4]:
filepath = glob('../../data/ddh_standard_frame_data/task_0421_p19_backup_2023_05_10_06_37_45/data/*.jpg')
filepath.sort()
save_path= [f.replace('/ddh_standard_frame_data/task_0421_p19_backup_2023_05_10_06_37_45/data', '/video/MobileNet') for f in filepath]
def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df
standard_file_path=[]
with open('../../data/ddh_standard_frame_data/task_0421_p19_backup_2023_05_10_06_37_45/annotations.json') as f:
    annotations_json = json.load(f)
std_frame=AnnotationsPd(annotations_json[0])['frame']

In [7]:

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


for i in range(len(filepath)):
    
    image=Image.open(filepath[i])
    dst_image=copy(image)
    start = time.time()
    image=image.crop((196,128,508,640))
    image=expand2square(image,(0,0,0))
    image=image.resize((224,224))
    image=np.array(image).reshape(1,224,224,3)/255
    ground_truth=(std_frame==i+1).any()
  
    pred=model.predict(image)
    end = time.time()
    draw = ImageDraw.Draw(dst_image)
    if pred>0.854:
        text="standardFrame"+str(pred)
        fill1="green"
        draw.rectangle((0,0,dst_image.size[0],dst_image.size[1]), outline=(0,255,0), width = 3)
    else :
        text="Not standardFrame"+str(pred)
        fill1="red"
        draw.rectangle((0,0,dst_image.size[0],dst_image.size[1]), outline=(255,0,0), width = 3)
        
    radius = 10
    if ground_truth:
        draw.ellipse((20 - radius,60 - radius, 20 + radius, 60 + radius), fill=(0, 255, 0))
    else:
        draw.ellipse((20 - radius,60 - radius, 20 + radius, 60 + radius), fill=(255, 0, 0))
    draw.text((10,10), text, font=font, fill='white')
    draw.text((10,25), f"{end - start:.5f} sec", font=font, fill='white')
    draw = ImageDraw.Draw(dst_image)
    dst_image.save(save_path[i])

1/1 [==============================] - 0s 34ms/step


In [ ]:
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)
print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)